셀0 : 패키지 설치 및 conda 환경설정

# 콘다 프롬프트(터미널)에서 실행
conda create -n hecto_car python=3.10 -y
conda activate hecto_car

# Jupyter 노트북 커널 연결용 ipykernel 설치
pip install ipykernel
python -m ipykernel install --user --name hecto_car --display-name "hecto_car"

In [1]:
# # 셀 0 ─ Windows 호환 패키지 설치
# import sys, subprocess, re, importlib, platform

# INSTALL_FAISS_GPU = False   # ← GPU 버전 쓰려면 True 로 바꾸고 conda 사용

# def clean(pkg): return re.split(r'[<=>]', pkg)[0]

# def pip_install(pkg):
#     subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "--quiet"])

# required = [
#     "timm>=0.9.12",
#     "albumentations>=1.4.0",
#     "einops>=0.7.0",
#     "pandas",
#     "scikit-learn",
#     "tqdm",
# ]

# # 1) 일반 패키지
# for p in required:
#     name = clean(p)
#     if importlib.util.find_spec(name) is None:
#         print("⏳", p)
#         pip_install(p)
#     else:
#         print("✅", name, "already")

# # 2) FAISS 처리
# if INSTALL_FAISS_GPU and platform.system() != "Windows":
#     try:
#         pip_install("faiss-gpu>=1.7.2")
#     except subprocess.CalledProcessError:
#         print("⚠️  pip wheel not found, trying conda ...")
#         print("run in shell:  conda install -c conda-forge faiss-gpu==1.7.2")
#         raise
# else:
#     # CPU 버전
#     if importlib.util.find_spec("faiss") is None:
#         print("⏳ installing faiss-cpu")
#         pip_install("faiss-cpu>=1.7.2")
#     else:
#         print("✅ faiss already")

# import torch, timm, albumentations, faiss, einops, pandas, sklearn
# print("📦  All packages ready — PyTorch", torch.__version__)


In [2]:
# 셀 1 : 기본 경로 설정 & 시드 고정
import os, random, numpy as np, torch

# 절대 경로 (Windows)
ROOT       = r"C:\Users\shaun\Desktop\project\Daycon\hecto_car_classification"
TRAIN_DIR  = os.path.join(ROOT, "data", "train")
TEST_DIR   = os.path.join(ROOT, "data", "test")

CFG = dict(
    IMG_SIZE = 448,
    BATCH    = 32,
    EPOCH    = 20,
    LR       = 3e-4,
    FOLDS    = 5,
    SEED     = 42
)

def seed_everything(seed:int=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False

seed_everything(CFG["SEED"])

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧  ROOT  : {ROOT}")
print(f"🖼️   Train: {TRAIN_DIR}")
print(f"🖼️   Test : {TEST_DIR}")
print(f"🚀  Device: {device}  |  Seed: {CFG['SEED']}")


🔧  ROOT  : C:\Users\shaun\Desktop\project\Daycon\hecto_car_classification
🖼️   Train: C:\Users\shaun\Desktop\project\Daycon\hecto_car_classification\data\train
🖼️   Test : C:\Users\shaun\Desktop\project\Daycon\hecto_car_classification\data\test
🚀  Device: cpu  |  Seed: 42


In [3]:
# 셀 2 : 이미지 경로 -> DataFrame  + Stratified 5-Fold
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from collections import Counter

# 1) 폴더명 = 클래스명 목록
class_names = sorted([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])
cls2id = {c:i for i,c in enumerate(class_names)}

# 2) 모든 이미지 경로 수집
records = []
for cls in class_names:
    cls_dir = os.path.join(TRAIN_DIR, cls)
    for fname in os.listdir(cls_dir):
        if fname.lower().endswith(".jpg"):
            records.append([os.path.join(cls_dir, fname), cls2id[cls]])

df = pd.DataFrame(records, columns=["img_path", "label"])
print(f"총 이미지 수 : {len(df):,}  |  클래스 수 : {len(class_names)}")

# 3) Stratified K-Fold split
df["fold"] = -1
skf = StratifiedKFold(n_splits=CFG["FOLDS"], shuffle=True, random_state=CFG["SEED"])

for fold, (_, val_idx) in enumerate(skf.split(df, df["label"])):
    df.loc[val_idx, "fold"] = fold

# 4) 분포 확인
fold_sizes = df.groupby("fold").size()
cls_min_per_fold = df.groupby(["fold","label"]).size().groupby("label").min().min()

print("Fold 이미지 개수 :", fold_sizes.to_dict())
print("각 클래스가 모든 fold에 ≥1장 존재?  ➜", "Yes" if cls_min_per_fold > 0 else "⚠️  일부 fold에서 사라진 클래스 있음")

# (선택) 희귀 클래스 경고
rare = [c for c,cnt in Counter(df["label"]).items() if cnt < 10]
if rare:
    print(f"⚠️  샘플 <10장 클래스 {len(rare)}개 → Leave-One-Out 추가 고려")

df.head()


총 이미지 수 : 33,137  |  클래스 수 : 396
Fold 이미지 개수 : {0: 6628, 1: 6628, 2: 6627, 3: 6627, 4: 6627}
각 클래스가 모든 fold에 ≥1장 존재?  ➜ Yes


,img_path,label,fold
0,C:\Users\shaun\Desktop\project\Daycon\hecto_ca...,0,4
1,C:\Users\shaun\Desktop\project\Daycon\hecto_ca...,0,0
2,C:\Users\shaun\Desktop\project\Daycon\hecto_ca...,0,4
3,C:\Users\shaun\Desktop\project\Daycon\hecto_ca...,0,1
4,C:\Users\shaun\Desktop\project\Daycon\hecto_ca...,0,0


In [8]:
# 셀 3 ─ Albumentations 변형 (모듈 미존재 이슈 없는 최소 + 강 증강)
import albumentations as A
from albumentations.pytorch import ToTensorV2

MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]
IMG  = CFG["IMG_SIZE"]

def get_transforms(phase="train"):
    if phase == "train":
        return A.Compose([
            # 기본 크롭
            A.RandomResizedCrop(size=(IMG, IMG), scale=(0.6, 1.0), ratio=(0.75, 1.333)),
            # ── 강증강 대체: ColorJitter or ShiftScaleRotate 중 하나 무조건 적용
            A.OneOf([
                A.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1, p=1.0),
                A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=1.0),
            ], p=1.0),
            A.HorizontalFlip(p=0.5),
            # 부분 가림
            A.CoarseDropout(max_holes=1, max_height=int(IMG*0.25), max_width=int(IMG*0.25),
                            min_holes=1, min_height=int(IMG*0.1),  min_width=int(IMG*0.1),
                            fill_value=0, p=0.3),
            A.Normalize(mean=MEAN, std=STD),
            ToTensorV2(),
        ])
    else:                           # validation / inference
        return A.Compose([
            A.Resize(height=IMG, width=IMG),
            A.Normalize(mean=MEAN, std=STD),
            ToTensorV2(),
        ])

train_tf = get_transforms("train")
val_tf   = get_transforms("val")
print("✅ Albumentations transforms ready (basic + strong jitter).")


✅ Albumentations transforms ready (basic + strong jitter).


c:\Users\shaun\anaconda3\envs\hecto_car\lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\shaun\AppData\Local\Temp\ipykernel_23396\720496036.py:21: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=1, max_height=int(IMG*0.25), max_width=int(IMG*0.25),


In [9]:
# 셀 4 — Custom Dataset + CutMix Collate
# --------------------------------------
import cv2
import torch
from torch.utils.data import Dataset
import numpy as np


class CarDataset(Dataset):
    """
    df : pd.DataFrame with columns ['img_path', 'label', 'fold']
    transform : Albumentations object (train_tf / val_tf)
    """
    def __init__(self, df, transform=None, is_test=False):
        self.paths = df["img_path"].tolist()
        self.labels = None if is_test else df["label"].tolist()
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            img = self.transform(image=img)["image"]  # returns torch.Tensor [C,H,W]

        if self.is_test:
            return img, self.paths[idx]               # path 보존
        else:
            label = self.labels[idx]
            return img, label


# ---------- CutMix Collate --------------------------------------------------
def rand_bbox(W, H, lam):
    """returns bbx1, bby1, bbx2, bby2"""
    cut_rat = np.sqrt(1. - lam)
    cut_w, cut_h = int(W * cut_rat), int(H * cut_rat)
    # uniform center
    cx, cy = np.random.randint(W), np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2


def get_cutmix_collate(n_classes, alpha=1.0, prob=0.5):
    """
    factory → DataLoader(collate_fn=...) 로 넘긴다
    labels → soft-target one-hot Tensor (B, n_classes)
    """
    def _collate(batch):
        imgs, labels = list(zip(*batch))
        imgs = torch.stack(imgs)
        labels = torch.tensor(labels)

        onehot = torch.zeros(imgs.size(0), n_classes, dtype=torch.float32)
        onehot.scatter_(1, labels.view(-1, 1), 1.0)

        if np.random.rand() < prob:
            lam = np.random.beta(alpha, alpha)
            rand_idx = torch.randperm(imgs.size(0))
            shuffled_imgs = imgs[rand_idx]
            shuffled_onehot = onehot[rand_idx]

            _, H, W = imgs.shape[1:]
            bbx1, bby1, bbx2, bby2 = rand_bbox(W, H, lam)

            imgs[:, :, bby1:bby2, bbx1:bbx2] = shuffled_imgs[:, :, bby1:bby2, bbx1:bbx2]
            lam_adj = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (W * H))
            onehot = onehot * lam_adj + shuffled_onehot * (1. - lam_adj)

        return imgs, onehot
    return _collate


# ---------- DataLoader 예시 (train / val) -----------------------------------
n_classes = len(class_names)  # 셀 2에서 확보

train_df = df[df.fold != 0].reset_index(drop=True)   # 예: fold0 를 검증으로
val_df   = df[df.fold == 0].reset_index(drop=True)

train_set = CarDataset(train_df, transform=train_tf)
val_set   = CarDataset(val_df,   transform=val_tf)

cutmix_collate = get_cutmix_collate(n_classes, alpha=1.0, prob=0.5)

from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=CFG["BATCH"],
                          shuffle=True, num_workers=4,
                          collate_fn=cutmix_collate)

val_loader   = DataLoader(val_set, batch_size=CFG["BATCH"],
                          shuffle=False, num_workers=4,
                          collate_fn=lambda b: (torch.stack([x[0] for x in b]),
                                                torch.tensor([x[1] for x in b])))

print(f"✅ DataLoader ready  |  train={len(train_set)}  val={len(val_set)}  classes={n_classes}")


✅ DataLoader ready  |  train=26509  val=6628  classes=396


In [10]:
# 셀 5 ─ make_loaders 함수: Fold 인덱스 → train / val / test DataLoader
from torch.utils.data import DataLoader

def make_loaders(fold:int,
                 df_full,
                 batch_size:int = CFG["BATCH"],
                 num_workers:int = 4):
    """
    fold : 검증으로 지정할 fold 번호 (0~CFG['FOLDS']-1)
    df_full : 셀 2에서 만든 DataFrame (img_path, label, fold)
    반환값 : train_loader, val_loader, test_loader
    """
    # 1) 분할
    train_df = df_full[df_full.fold != fold].reset_index(drop=True)
    val_df   = df_full[df_full.fold == fold].reset_index(drop=True)

    # 2) Dataset
    train_set = CarDataset(train_df, transform=train_tf)
    val_set   = CarDataset(val_df,   transform=val_tf)
    
    # 3) Collate
    cutmix_fn = get_cutmix_collate(n_classes=len(class_names), alpha=1.0, prob=0.5)

    train_loader = DataLoader(
        train_set,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=cutmix_fn
    )

    # 검증은 CutMix 없음 → 이미지 스택 + 정수 라벨 Tensor
    val_loader = DataLoader(
        val_set,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=lambda batch: (
            torch.stack([b[0] for b in batch]),
            torch.tensor([b[1] for b in batch])
        )
    )

    # 4) Test Loader (is_test=True)
    test_paths = sorted([os.path.join(TEST_DIR, f)
                         for f in os.listdir(TEST_DIR)
                         if f.lower().endswith(".jpg")])
    test_df = pd.DataFrame({"img_path": test_paths})
    test_set = CarDataset(test_df, transform=val_tf, is_test=True)

    test_loader = DataLoader(
        test_set,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=lambda batch: (
            torch.stack([b[0] for b in batch]),   # images
            [b[1] for b in batch]                 # img_path list (ID)
        )
    )

    print(f"📊 Fold {fold}  |  train={len(train_set)}  val={len(val_set)}  test={len(test_set)}")
    return train_loader, val_loader, test_loader


# 예시: fold 0 로더 생성
train_loader, val_loader, test_loader = make_loaders(fold=0, df_full=df)


📊 Fold 0  |  train=26509  val=6628  test=8258


In [11]:
# 셀 6 ─ GeM + Sub-center ArcFace Head
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from einops import rearrange

# ---------------- GeM Pool ---------------------------------------------------
class GeM(nn.Module):
    def __init__(self, p: float = 3.0, eps: float = 1e-6):
        super().__init__()
        self.p   = nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        # x : (B,C,H,W)
        return F.avg_pool2d(x.clamp(min=self.eps).pow(self.p),
                            kernel_size=(x.size(-2), x.size(-1))
                           ).pow(1.0 / self.p).flatten(1)

# ---------------- Sub-center ArcFace ----------------------------------------
class SubCenterArcFace(nn.Module):
    """
    Implements Sub-center ArcFace
    * in_features  : backbone feature dim
    * out_classes  : number of classes
    * k            : sub-centers per class
    * s            : scale factor (logits = cos * s)
    """
    def __init__(self, in_features: int, out_classes: int, k: int = 3, s: float = 30.0):
        super().__init__()
        self.out_classes = out_classes
        self.k           = k
        self.s           = s

        # weight shape : [out_classes * k, in_features]
        self.weight = nn.Parameter(torch.randn(out_classes * k, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, x):
        # x  : (B, in_features)        – assume already flattened
        x = F.normalize(x, dim=1)
        W = F.normalize(self.weight, dim=1)

        # cosine sim: (B, out_classes * k)
        cos = F.linear(x, W)

        # reshape to (B, classes, k)  → 최대 sub-center 선택
        cos = rearrange(cos, "b (c k) -> b c k", c=self.out_classes, k=self.k)
        cos_max, _ = torch.max(cos, dim=2)          # (B, classes)

        logits = cos_max * self.s
        return logits

# ---------------- Backbone + Head -------------------------------------------
class CarNet(nn.Module):
    def __init__(self, n_classes: int, k: int = 3):
        super().__init__()
        self.backbone = timm.create_model(
            "convnext_base", pretrained=True, num_classes=0   # feature only
        )
        in_dim = self.backbone.num_features
        self.pool = GeM(p=3)
        self.head = SubCenterArcFace(in_dim, n_classes, k=k, s=30.0)

    def forward(self, x):
        feat = self.backbone.forward_features(x)    # (B,C,H,W)
        feat = self.pool(feat)                      # (B,C)
        logits = self.head(feat)                    # (B, n_classes)
        return logits

# 예시 인스턴스
n_classes = len(class_names)       # 396
model = CarNet(n_classes=n_classes, k=3).to(device)
print("✅ Model initialized – ConvNeXt-B + GeM + Sub-center ArcFace")


✅ Model initialized – ConvNeXt-B + GeM + Sub-center ArcFace


c:\Users\shaun\anaconda3\envs\hecto_car\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shaun\.cache\huggingface\hub\models--timm--convnext_base.fb_in22k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [12]:
# 셀 7 ─ 학습 세트업 (Loss · Optim · Scheduler)

# 1) 모델 이미 셀 6에서 생성
#    model = CarNet(n_classes=len(class_names), k=3).to(device)

# 2) Loss   (CutMix → soft one-hot target)
criterion = nn.BCEWithLogitsLoss()

# 3) Optimizer
#    - Backbone에 기본 LR (3e-4), Head 파라미터에 10× LR
def param_groups(model, base_lr, head_lr_mul=10):
    backbone, head = [], []
    for n, p in model.named_parameters():
        (head if "head" in n else backbone).append(p)
    return [
        {"params": backbone, "lr": base_lr},
        {"params": head,     "lr": base_lr * head_lr_mul},
    ]

optimizer = torch.optim.AdamW(
    param_groups(model, CFG["LR"]),
    lr=CFG["LR"],
    weight_decay=1e-2
)

# 4) Scheduler  (Cosine with warm-up 3 epochs)
warmup_epochs = 3
total_steps   = CFG["EPOCH"] * len(train_loader)
warmup_steps  = warmup_epochs * len(train_loader)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=total_steps - warmup_steps
)

def lr_lambda(step):
    if step < warmup_steps:
        return step / warmup_steps
    return 1.0
warmup_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# 5) AMP & EMA (선택)
scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))
ema_decay = 0.999
ema_weights = [p.clone().detach() for p in model.parameters()]

print("✅ Training components ready – BCEWithLogitsLoss / AdamW + CosineLR")


✅ Training components ready – BCEWithLogitsLoss / AdamW + CosineLR


C:\Users\shaun\AppData\Local\Temp\ipykernel_23396\3664142599.py:42: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))


In [ ]:
# 셀 8 ─ Train / Validate Loop (AMP + Warm-up + EMA)

from sklearn.metrics import log_loss
import math, copy, time
from tqdm.auto import tqdm


def one_hot(label_tensor, num_cls):
    """labels(long) -> one-hot(float32)"""
    y = torch.zeros((label_tensor.size(0), num_cls), device=label_tensor.device)
    y.scatter_(1, label_tensor.view(-1, 1), 1.0)
    return y


def update_ema(model, ema_w, decay):
    with torch.no_grad():
        for p, e in zip(model.parameters(), ema_w):
            e.mul_(decay).add_(p.data, alpha=1.0 - decay)


best_val_loss = math.inf
oof_logits, oof_labels = [], []

global_step = 0
for epoch in range(CFG["EPOCH"]):
    # ---------- Train -------------------------------------------------------
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{CFG['EPOCH']} Train", leave=False)
    running_loss = 0.0

    for imgs, soft_targets in pbar:
        imgs = imgs.to(device, non_blocking=True)
        soft_targets = soft_targets.to(device)

        with torch.cuda.amp.autocast(enabled=(device == "cuda")):
            logits = model(imgs)
            loss = criterion(logits, soft_targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)

        # LR scheduler (warm-up + cosine)
        if global_step < warmup_steps:
            warmup_scheduler.step()
        else:
            scheduler.step()
        global_step += 1

        # EMA
        update_ema(model, ema_weights, ema_decay)

        running_loss += loss.item()
        pbar.set_postfix(loss=f"{running_loss/ (pbar.n):.4f}")

    # ---------- Validation --------------------------------------------------
    model.eval()
    val_losses, val_probs, val_lbls = [], [], []
    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc="Valid", leave=False):
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device)

            logits = model(imgs)
            targets = one_hot(labels, n_classes)
            loss = criterion(logits, targets)
            val_losses.append(loss.item())

            probs = torch.sigmoid(logits).cpu().numpy()
            val_probs.append(probs)
            val_lbls.append(labels.cpu().numpy())

    val_loss = np.mean(val_losses)
    val_probs = np.concatenate(val_probs)
    val_lbls  = np.concatenate(val_lbls)
    # log_loss expects probabilities for each class (not softmax due to BCE)
    val_logloss = log_loss(val_lbls, val_probs, labels=list(range(n_classes)))

    print(f"Epoch {epoch+1:02d}  TrainLoss {running_loss/len(train_loader):.4f} |"
          f" ValLoss {val_loss:.4f} | LogLoss {val_logloss:.4f}")

    # save best
    if val_logloss < best_val_loss:
        best_val_loss = val_logloss
        torch.save({"model": model.state_dict(),
                    "ema":  [w.cpu() for w in ema_weights]},
                   f"{ROOT}\\best_model_fold0.pth")
        print(f"  ✅ best model saved (LogLoss {best_val_loss:.4f})")

    # accumulate OOF for optional later use
    oof_logits.append(val_probs)
    oof_labels.append(val_lbls)


Epoch 1/20 Train:   0%|          | 0/829 [00:00<?, ?it/s]c:\Users\shaun\anaconda3\envs\hecto_car\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
